# Deploying a Chatbot to the Web
* **Created by:** Eric Martinez
* **For:** Software Engineering 2
* **At:** University of Texas Rio-Grande Valley

## HuggingFace

HuggingFace is an AI research organization and platform that provides access to a wide range of pre-trained LLMs and tools for training, fine-tuning, and deploying models. It has a user-friendly interface and a large community, making it a popular choice for working with LLMs.

## Deploying to HuggingFace

#### Configuring the files required

Let's face it! Once we start building cool stuff we are going to want to show it off. It can take us < 10 minutes to deploy our chatbots and LLM applications when using Gradio!

Add a username and password for your app to your `.env` file. This will ensure that unauthorized users are not able to access LLM features. Use the following format:

In [ ]:
APP_USERNAME=<whatever username you want>
APP_PASSWORD=<whatever password you want>

Let's start by taking all of our necessary chatbot code into one file which we will name `app.py`. Run the following cell to automatically write it!

Take note that this code has been altered a little bit from the last chatbot example in order to add authentication.

In [12]:
%%writefile app.py
import gradio as gr
import openai
import examples as chatbot_examples
from dotenv import load_dotenv
import os

load_dotenv()  # take environment variables from .env.

# In order to authenticate, secrets must have been set, and the user supplied credentials match
def auth(username, password):
    app_username = os.getenv("APP_USERNAME")
    app_password = os.getenv("APP_PASSWORD")

    if app_username and app_password:
        if(username == app_username and password == app_password):
            print("Logged in successfully.")
            return True
        else:
            print("Username or password does not match.")
    else:
        print("Credential secrets not set.")
    return False
    
# Define a function to get the AI's reply using the OpenAI API
def get_ai_reply(message, model="gpt-3.5-turbo", system_message=None, temperature=0, message_history=[]):
    # Initialize the messages list
    messages = []
    
    # Add the system message to the messages list
    if system_message is not None:
        messages += [{"role": "system", "content": system_message}]

    # Add the message history to the messages list
    if message_history is not None:
        messages += message_history
    
    # Add the user's message to the messages list
    messages += [{"role": "user", "content": message}]
    
    # Make an API call to the OpenAI ChatCompletion endpoint with the model and messages
    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    
    # Extract and return the AI's response from the API response
    return completion.choices[0].message.content.strip()

# Define a function to handle the chat interaction with the AI model
def chat(model, system_message, message, chatbot_messages, history_state):
    # Initialize chatbot_messages and history_state if they are not provided
    chatbot_messages = chatbot_messages or []
    history_state = history_state or []
    
    # Try to get the AI's reply using the get_ai_reply function
    try:
        ai_reply = get_ai_reply(message, model=model, system_message=system_message, message_history=history_state)
    except Exception as e:
        # If an error occurs, raise a Gradio error
        raise gr.Error(e)
    
    # Append the user's message and the AI's reply to the chatbot_messages list
    chatbot_messages.append((message, ai_reply))
    
    # Append the user's message and the AI's reply to the history_state list
    history_state.append({"role": "user", "content": message})
    history_state.append({"role": "assistant", "content": ai_reply})
    
    # Return None (empty out the user's message textbox), the updated chatbot_messages, and the updated history_state
    return None, chatbot_messages, history_state

# Define a function to launch the chatbot interface using Gradio
def get_chatbot_app(additional_examples=[]):
    # Load chatbot examples and merge with any additional examples provided
    examples = chatbot_examples.load_examples(additional=additional_examples)
    
    # Define a function to get the names of the examples
    def get_examples():
        return [example["name"] for example in examples]

    # Define a function to choose an example based on the index
    def choose_example(index):
        if(index!=None):
            system_message = examples[index]["system_message"].strip()
            user_message = examples[index]["message"].strip()
            return system_message, user_message, [], []
        else:
            return "", "", [], []

    # Create the Gradio interface using the Blocks layout
    with gr.Blocks() as app:
        with gr.Tab("Conversation"):
            with gr.Row():
                with gr.Column():
                    # Create a dropdown to select examples
                    example_dropdown = gr.Dropdown(get_examples(), label="Examples", type="index")
                    # Create a button to load the selected example
                    example_load_btn = gr.Button(value="Load")
                    # Create a textbox for the system message (prompt)
                    system_message = gr.Textbox(label="System Message (Prompt)", value="You are a helpful assistant.")
                with gr.Column():
                    # Create a dropdown to select the AI model
                    model_selector = gr.Dropdown(
                        ["gpt-3.5-turbo"],
                        label="Model",
                        value="gpt-3.5-turbo"
                    )
                    # Create a chatbot interface for the conversation
                    chatbot = gr.Chatbot(label="Conversation")
                    # Create a textbox for the user's message
                    message = gr.Textbox(label="Message")
                    # Create a state object to store the conversation history
                    history_state = gr.State()
                    # Create a button to send the user's message
                    btn = gr.Button(value="Send")

                # Connect the example load button to the choose_example function
                example_load_btn.click(choose_example, inputs=[example_dropdown], outputs=[system_message, message, chatbot, history_state])
                # Connect the send button to the chat function
                btn.click(chat, inputs=[model_selector, system_message, message, chatbot, history_state], outputs=[message, chatbot, history_state])
        # Return the app
        return app
        
# Call the launch_chatbot function to start the chatbot interface using Gradio
# Set the share parameter to False, meaning the interface will not be publicly accessible
app = get_chatbot_app()
app.queue()  # this is to be able to queue multiple requests at once
app.launch(auth=auth)

Overwriting app.py


We will also need a `requirements.txt` file to store the list of the packages that HuggingFace needs to install to run our chatbot.

In [13]:
%%writefile requirements.txt
gradio == 3.27.0
openai == 0.27.4
python-dotenv == 1.0.0

Overwriting requirements.txt


Now let's go ahead and commit our changes

In [9]:
!git add app.py

In [8]:
!git add requirements.txt

In [11]:
!git commit -m "adding chatbot"

[master (root-commit) 39899ef] adding chatbot
 2 files changed, 101 insertions(+)
 create mode 100644 app.py
 create mode 100644 requirements.txt


#### Using HuggingFace Spaces

As mentioned before, HuggingFace is a free-to-use platform for hosting AI demos and apps. We will need to make a HuggingFace _Space_ for our chatbot.

First sign up for a free HuggingFace account [here](https://huggingface.co/join). After you sign up, create a new Space by clicking "New Space" on the navigation menu (press on your profile image).

#### Generate a HuggingFace Access Token

#### Login to HuggingFace Hub

Install `huggingface_hub`

In [40]:
!pip -q install --upgrade huggingface_hub

Login to HuggingFace

In [6]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (osxkeychain).
Your token has been saved to /Users/ericmartinez/.cache/huggingface/token
Login successful


#### Now lets setup git and HuggingFace Spaces to work together and deploy

<span style="color:red">REPLACE MY URL WITH YOURS</span>

In [ ]:
!git remote add huggingface https://huggingface.co/spaces/ericmichael/gradio-chatbot-demo

Then force push to sync everything for the first time.

In [12]:
!git push --force huggingface main

Total 0 (delta 0), reused 0 (delta 0), pack-reused 0
To https://huggingface.co/spaces/ericmichael/gradio-chatbot-demo
 + 8911ec0...3693bcc main -> main (forced update)


That's it! 🎉 Check your HuggingFace Space URL to access your chatbot!

## Deploying using FastAPI